<a href="https://colab.research.google.com/github/t0brig01/CSE696FinalProject/blob/main/MoralNeuralNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import pandas as pd
from pandas import concat
from sklearn import preprocessing
print ("TensorFlow version: " + tf.__version__)

TensorFlow 1.x selected.
TensorFlow version: 1.15.2


# **Data Loading/Manipulation**

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/t0brig01/CSE696FinalProject/main/TrolleyData.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230475 entries, 0 to 230474
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   _id             230475 non-null  object
 1   Scenario        230475 non-null  object
 2   UserID          230475 non-null  object
 3   Outcome         230475 non-null  int64 
 4   Session_id      230475 non-null  object
 5   Scenario_order  230475 non-null  int64 
 6   Template        230475 non-null  object
 7   answerLeft      230475 non-null  bool  
 8   lang            230475 non-null  object
 9   seenOther       230475 non-null  bool  
 10  country_code    229259 non-null  object
 11  country_full    229259 non-null  object
dtypes: bool(2), int64(2), object(8)
memory usage: 18.0+ MB


In [5]:
df.head()

,_id,Scenario,UserID,Outcome,Session_id,Scenario_order,Template,answerLeft,lang,seenOther,country_code,country_full
0,C3RGCjGPftjMKYHy2,Loop,0000dc12_9518522259818270,1,367585191-9.51852225982e+15,1,Desktop,False,de,True,DE,Germany
1,3F9SfDyofskDPna8R,Footbridge,0000dc12_9518522259818270,0,367585191-9.51852225982e+15,2,Desktop,False,de,True,DE,Germany
2,oz9anPbE6e52TKuJy,Switch,0000dc12_9518522259818270,1,367585191-9.51852225982e+15,3,Desktop,False,de,True,DE,Germany
3,dkAhBaLhzi62Pp6Hp,Footbridge,0002ae2d_2286850331484848,0,1654499857-2.28685033148e+15,1,Desktop,False,en,True,GB,United Kingdom
4,2ussgtw7vCZMNjxSJ,Switch,0002ae2d_2286850331484848,1,1654499857-2.28685033148e+15,2,Desktop,False,en,True,GB,United Kingdom


In [64]:
filtered = df.drop(['Session_id','Template','country_full','_id','answerLeft'], axis=1)
filtered.dropna()
filtered.replace('Loop',0,inplace=True)   #0 is loop trolley 
filtered.replace('Footbridge',1,inplace=True) #1 is footbridge trolley
filtered.replace('Switch',2,inplace=True) #2 is standard trolley

lang = dict()
country = dict()
iLang = 0
iCountry = 0
for index, x in filtered.iterrows():
  if x["lang"] not in lang.keys():
    lang[x["lang"]] = iLang
    iLang = iLang + 1
  if x["country_code"] not in country.keys():
    country[x["country_code"]] = iCountry
    iCountry = iCountry + 1
for key, value in country.items():
  filtered.replace(key,int(value),inplace=True)
for key, value in lang.items():
  filtered.replace(key,int(value),inplace=True)

,Scenario,UserID,Outcome,Scenario_order,lang,seenOther,country_code
0,0,0000dc12_9518522259818270,1,1,0,True,0
1,1,0000dc12_9518522259818270,0,2,0,True,0
2,2,0000dc12_9518522259818270,1,3,0,True,0
3,1,0002ae2d_2286850331484848,0,1,1,True,1
4,2,0002ae2d_2286850331484848,1,2,1,True,1
...,...,...,...,...,...,...,...
230470,1,NA_9906217589049584,0,3,3,True,169
230471,2,NA_991789531797033,1,1,3,False,169
230472,1,NA_9957209681632944,1,1,1,True,169
230473,2,NA_9957209681632944,0,2,1,True,169


In [68]:
filtered['country_code'] = filtered['country_code'].astype(int)

In [69]:
filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230475 entries, 0 to 230474
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Scenario        230475 non-null  int64 
 1   UserID          230475 non-null  object
 2   Outcome         230475 non-null  int64 
 3   Scenario_order  230475 non-null  int64 
 4   lang            230475 non-null  int64 
 5   seenOther       230475 non-null  bool  
 6   country_code    230475 non-null  int64 
dtypes: bool(1), int64(5), object(1)
memory usage: 10.8+ MB


In [71]:
def create_dataset(X):
  Xs = X.drop(["UserID","Outcome"],axis = 1)
  Ys = X.drop(["Scenario","UserID","Scenario_order","lang","seenOther","country_code"],axis=1)
  return np.array(Xs),np.array(Ys)

In [72]:
interval = int(len(filtered.index)/3)
x_train, y_train = create_dataset(filtered.head(interval))
x_test, y_test = create_dataset(filtered.tail(interval))
x_val, y_val = create_dataset(filtered.loc[interval:interval*2])

# **Neural Network**

In [74]:
from keras.models import Sequential
from keras import layers
from keras.layers import LSTM, Dense
from keras.optimizers import RMSprop,SGD
from tensorflow import keras
from keras import optimizers
import tensorflow as tf


model = Sequential()
model.add(Dense(60, input_dim=5, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train,y_train,epochs=30,validation_data=(x_val,y_val))

Train on 76825 samples, validate on 76826 samples
Epoch 1/30
76825/76825 [==============================] - 6s 73us/step - loss: 0.5957 - accuracy: 0.6971 - val_loss: 0.6141 - val_accuracy: 0.6992
Epoch 2/30
76825/76825 [==============================] - 6s 73us/step - loss: 0.5830 - accuracy: 0.7005 - val_loss: 0.5798 - val_accuracy: 0.7020
Epoch 3/30
76825/76825 [==============================] - 6s 74us/step - loss: 0.5803 - accuracy: 0.6997 - val_loss: 0.5830 - val_accuracy: 0.7012
Epoch 4/30
76825/76825 [==============================] - 6s 73us/step - loss: 0.5792 - accuracy: 0.7015 - val_loss: 0.5806 - val_accuracy: 0.7005
Epoch 5/30
76825/76825 [==============================] - 5s 71us/step - loss: 0.5785 - accuracy: 0.7020 - val_loss: 0.5843 - val_accuracy: 0.6994
Epoch 6/30
76825/76825 [==============================] - 5s 71us/step - loss: 0.5782 - accuracy: 0.7013 - val_loss: 0.5796 - val_accuracy: 0.7014
Epoch 7/30
76825/76825 [==============================] - 6s 72us/st

In [31]:
filtered.groupby(["lang"]).size()

lang
ar       304
de     17292
en    139896
es     10356
fr     23045
ja       729
kr       670
pt     14760
ru     14608
zh      8815
dtype: int64

In [33]:
filtered.groupby(["country_code"]).size()

country_code
-      269
AD      32
AE     324
AF      12
AL      15
      ... 
VI       3
VN    1337
YT       9
ZA     438
ZM       6
Length: 169, dtype: int64